In [58]:
# Chargement des bibliothèques nécessaires
import nltk
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [59]:
# Préparation des données d'entraînement
# Chaque élément de la liste est une phrase avec des étiquettes indiquant
# les points de départ et d'arrivée
data = [
    {
        'sentence': 'Je vais de Paris à Londres en train.',
        'start': 'Paris',
        'end': 'Londres'
    },
    {
        'sentence': 'Le vol de New York à Los Angeles dure environ cinq heures.',
        'start': 'New York',
        'end': 'Los Angeles'
    },
    {
        'sentence': 'Le voyage de Rome à Berlin prend environ sept heures en voiture.',
        'start': 'Rome',
        'end': 'Berlin'
    },
    {
        'sentence': 'Je veux allez à Nancy et je suis a Paris.',
        'start': 'Paris',
        'end': 'Nancy'
    },
    {
        'sentence': 'Je suis à Metz et je veux allez à Marseille',
        'start': 'Metz',
        'end': 'Marseille'
    },
    {
        'sentence': 'Je veux allez à Strasbourg depuis Nice',
        'start': 'Nice',
        'end': 'Strasbourg'
    }
]

In [60]:
# Séparation des données en ensembles d'entraînement et de test
train_data, test_data = train_test_split(data, test_size=0.2)

In [61]:
# Création de l'objet Tokenizer pour convertir les phrases en séquences de mots
tokenizer = Tokenizer()
tokenizer.fit_on_texts([sent['sentence'] for sent in train_data])

In [62]:
# Conversion des phrases en séquences de mots
train_sequences = tokenizer.texts_to_sequences([sent['sentence'] for sent in train_data])
test_sequences = tokenizer.texts_to_sequences([sent['sentence'] for sent in test_data])

In [63]:
# Mise en forme des séquences pour qu'elles aient toutes la même longueur
max_length = max([len(seq) for seq in train_sequences + test_sequences])
train_sequences = pad_sequences(train_sequences, maxlen=max_length)
test_sequences = pad_sequences(test_sequences, maxlen=max_length)

In [64]:
# Création des modèles de réseau de neurones
start_model = tf.keras.Sequential()

# Ajout d'une couche d'embedding pour transformer les mots en vecteurs de nombres réels
start_model.add(tf.keras.layers.Embedding(len(tokenizer.word_index) + 1, 64, input_length=max_length))

# Ajout d'une couche de LSTM pour traiter les séquences de mots
start_model.add(tf.keras.layers.LSTM(64))

# Ajout d'une couche dense pour prédire la sortie
start_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compilation du modèle
start_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [65]:

# Entraînement du modèle
start_model.fit(train_sequences, [sent['start'] for sent in train_data], epochs=5, validation_split=0.2)

ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>]

In [ ]:
# Évaluation du modèle sur les données de test
start_predictions = start_model.predict(test_sequences)
start_accuracy = nltk.accuracy(start_predictions, [sent['start'] for sent in test_data])
print('Exactitude pour les points de départ : %.2f' % start_accuracy)

In [ ]:
# Répéter les étapes ci-dessus pour entraîner et évaluer le modèle pour les points d'arrivée
end_model = tf.keras.Sequential()
end_model.add(tf.keras.layers.Embedding(len(tokenizer.word_index) + 1, 64, input_length=max_length))
end_model.add(tf.keras.layers.LSTM(64))
end_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
end_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
end_model.fit(train_sequences, [sent['end'] for sent in train_data], epochs=5, validation_split=0.2)
end_predictions = end_model.predict(test_sequences)
end_accuracy = nltk.accuracy(end_predictions, [sent['end'] for sent in test_data])
print('Exactitude pour les points d\'arrivée : %.2f' % end_accuracy)